This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [93]:
# get api key from your .env file
import os
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [94]:
# First, import the relevant modules
import requests
import json
import numpy as np



Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [95]:
#Step 1
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#url = 'https://www.zeiss.com/meditec/int/home.html/?apikey={}&t=AFX_X'.format(API_KEY)
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key': API_KEY,    
          'start_date': '2017-01-01',
          'end_date': '2017-12-31'}
r = requests.get(url, params=params)
#Step 2 : Convert the returned JSON object into a Python dictionary.
json_data = r.json()

In [96]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [97]:
# See what the columns are there
column_names = json_data['dataset_data']['column_names']
data = json_data['dataset_data']['data']
print(column_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [98]:
# Step3:Calculate what the highest and lowest opening prices were for the stock in this period.
#load open price
#find the index
ind_open = column_names.index('Open')
opening_prices = [row[ind_open] for row in data if row[ind_open] is not None]
highest_opening_price = max(opening_prices)
lowest_opening_price = min(opening_prices)
print('Hihest and lowest opening price are {} and {} respetively'.format(highest_opening_price,lowest_opening_price))

Hihest and lowest opening price are 53.11 and 34.0 respetively


In [99]:
# Step 4 : What was the largest change in any one day (based on High and Low price)?
ind_high = column_names.index('High')
high_prices = [row[ind_high] for row in data if row[ind_high] is not None]
ind_low = column_names.index('Low')
low_prices = [row[ind_low] for row in data if row[ind_low] is not None]
largest_change_prices = max(list(set(high_prices) - set(low_prices)))
print('Largest change in any one day is {}'.format(largest_change_prices))


Largest change in any one day is 53.54


In [100]:
# Step 5 : What was the largest change between any two days (based on Closing Price)?
ind_close = column_names.index('Close')
closing_prices = [row[ind_close] for row in data if row[ind_close] is not None]
largest_change_prices_between_two_days = max([abs(closing_prices[i-1] - closing_prices[i]) for i in range(1, len(closing_prices))])
print('Largest change between any two days is {}'.format(largest_change_prices_between_two_days))

Largest change between any two days is 2.559999999999995


In [101]:
# Step 6 : What was the average daily trading volume during this year?
ind_trade = column_names.index('Traded Volume')
average_traded_volumn = np.mean([row[ind_trade] for row in data if row[ind_trade] is not None])
print('Averaged daily trading volume is {}'.format(average_traded_volumn))

Averaged daily trading volume is 89124.33725490196


In [102]:
# Step 7 : (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
sorted_traded_volumn = [row[ind_trade] for row in data if row[ind_trade] is not None]
len_sorted_traded_volumn = len(sorted_traded_volumn)
if len_sorted_traded_volumn%2 !=0:
    median_sorted_traded_volumn = sorted_traded_volumn[int((len_sorted_traded_volumn+1)/2)]
else:
    median_sorted_traded_volumn = sorted_traded_volumn(len_sorted_traded_volumn/2)
print('Median trading volume during this year is {}'.format(median_sorted_traded_volumn))

Median trading volume during this year is 117165.0
